In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_md')
doc = nlp(u'Hello, world. Here are two sentences.')

In [ ]:
texts = [u'One document.', u'...', u'Lots of documents']
# .pipe streams input, and produces streaming output
iter_texts = (texts[i % 3] for i in xrange(3))
for i, doc in enumerate(nlp.pipe(iter_texts, batch_size=50, n_threads=4)):
    assert doc.is_parsed
    if i == 0:
        break

In [ ]:
doc = nlp(u'Hello, world. Here are two sentences.')
token = doc[0]
sentence = next(doc.sents)
assert token is sentence[0]
assert sentence.text == 'Hello, world.'

In [ ]:
hello_id = nlp.vocab.strings['Hello']
hello_str = nlp.vocab.strings[hello_id]

assert token.orth  == hello_id  == 6747
assert token.orth_ == hello_str == 'Hello'

In [ ]:
from spacy.attrs import ORTH, LIKE_URL, IS_OOV

attr_ids = [ORTH, LIKE_URL, IS_OOV]
doc_array = doc.to_array(attr_ids)
assert doc_array.shape == (len(doc), len(attr_ids))
assert doc[0].orth == doc_array[0, 0]
assert doc[1].orth == doc_array[1, 0]
assert doc[0].like_url == doc_array[0, 1]
assert list(doc_array[:, 1]) == [t.like_url for t in doc]

In [ ]:
doc = nlp(u"Apples and oranges are similar. Boots and hippos aren't.")

apples = doc[0]
oranges = doc[2]
boots = doc[6]
hippos = doc[8]

assert apples.similarity(oranges) > boots.similarity(hippos)

In [ ]:
from spacy.parts_of_speech import ADV

def is_adverb(token):
    return token.pos == spacy.parts_of_speech.ADV
def is_adjective(token):
    return token.pos == spacy.parts_of_speech.ADJ

# These are data-specific, so no constants are provided. You have to look
# up the IDs from the StringStore.
NNPS = nlp.vocab.strings['similar']
sim = nlp.vocab.strings[NNPS]
def is_plural_noun(token):
    return token.tag == NNS or token.tag == NNPS

def print_coarse_pos(token):
    print(token.pos_)

def print_fine_pos(token):
    print(token.tag_)

similar = doc[4]
is_adjective(similar)
#is_adjective(sim)

In [ ]:
def dependency_labels_to_root(token):
    '''Walk up the syntactic tree, collecting the arc labels.'''
    dep_labels = []
    while token.head is not token:
        dep_labels.append(token.dep)
        token = token.head
    return dep_labels
root_id = dependency_labels_to_root(similar)
print(root_id)
print(nlp.vocab.strings[root_id[0]])

In [ ]:
hello_id = nlp.vocab.strings['Hello']
hello_str = nlp.vocab.strings[394]
print(hello_id)
print(hello_str)

In [ ]:
def put_spans_around_tokens(doc, get_classes):
    """Given some function to compute class names, put each token in a
    span element, with the appropriate classes computed. All whitespace is
    preserved, outside of the spans. (Of course, HTML won't display more than
    one whitespace character it – but the point is, no information is lost
    and you can calculate what you need, e.g. <br />, <p> etc.)
    """
    output = []
    html = '<span class="{classes}">{word}</span>{space}'
    for token in doc:
        if token.is_space:
            output.append(token.text)
        else:
            classes = ' '.join(get_classes(token))
            output.append(html.format(classes=classes, word=token.text, space=token.whitespace_))
    string = ''.join(output)
    string = string.replace('\n', '')
    string = string.replace('\t', '    ')
    return string

put_spans_around_tokens(doc, True)

In [ ]:
from spacy.tokens.doc import Doc

byte_string = doc.to_bytes()
open('moby_dick.bin', 'wb').write(byte_string)

for byte_string in Doc.read_bytes(open('moby_dick.bin', 'rb')):
    doc = Doc(nlp.vocab)
    doc.from_bytes(byte_string)

In [ ]:
doc